In [1]:
from utils.mapping2 import create_dataset, create_embedding_matrix,  load_all_features

import numpy as np
from tensorflow.keras import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

In [2]:
path = os.getcwd()
print(path)

r:\Project


In [3]:
train_feature_src = os.path.join(path,'dataset', 'features-small-train')
test_feature_src = os.path.join(path,'dataset', 'features-small-test')
train_data_src = os.path.join(path, 'dataset', 'MSVD_train-small.csv')


In [4]:
train_features, padded, target_padded, tokenizer, max_len = create_dataset(
    train_feature_src, train_data_src)

Creating Dataset...: 100%|██████████| 3177/3177 [00:02<00:00, 1391.18it/s]


In [5]:
train_features.shape, padded.shape, target_padded.shape, max_len

((3177, 20, 4096), (3177, 33), (3177, 33), 33)

In [6]:
test_feature, test_vid_ids = load_all_features(test_feature_src)
test_feature.shape, len(test_vid_ids)


Loading Features...: 100%|██████████| 20/20 [00:00<00:00, 445.53it/s]


((20, 20, 4096), 20)

In [7]:
vocab_size = len(tokenizer.word_index)
embd_vec_size = 100
glove_src = os.path.join(path,'dataset','glove.6B.100d.txt')

In [8]:
embd_matrix = create_embedding_matrix(glove_src, tokenizer.word_index, vocab_size, embd_vec_size)

Loading Word Embeddings....: 100%|██████████| 400001/400001 [00:24<00:00, 16421.83it/s]
[ 0.26688    0.39632    0.6169    -0.77451   -0.1039     0.26697
  0.2788     0.30992    0.0054685 -0.085256   0.73602   -0.098432
  0.5479    -0.030305   0.33479    0.14094   -0.0070003  0.32569
  0.22902    0.46557  ]
Embed not found for 151 words


In [9]:
embd_matrix.shape, vocab_size

((2140, 100), 2139)

In [10]:
lstm_units = 100
encoder_input_size = train_features.shape[2]
time_steps_enc = train_features.shape[1]

In [11]:
# Encoder

enc_input = Input((None, encoder_input_size), name='enc_input')
enc_lstm = LSTM(lstm_units, return_state=True, name='enc_lstm')
X = enc_input
X = enc_lstm(X)
enc_out, enc_h, enc_c = X
enc_states = [enc_h, enc_c]


In [12]:
# Decoder

dec_input = Input((None,), name='dec_input')
dec_embedding = Embedding(vocab_size+1, embd_vec_size, input_length=max_len,
                          trainable=False, name='dec_embd', mask_zero=True)
dec_lstm = LSTM(lstm_units, return_sequences=True,
                return_state=True, name='dec_lstm')
dec_dense = Dense(vocab_size+1, activation='softmax')
Y = dec_input
Y = dec_embedding(Y)
Y = dec_lstm(Y, initial_state=enc_states)
dec_out, _, _ = Y
Y = dec_dense(dec_out)

In [13]:
# Model

model = Model(inputs=[enc_input, dec_input], outputs=Y)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [14]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dec_input (InputLayer)          [(None, None)]       0                                            
__________________________________________________________________________________________________
enc_input (InputLayer)          [(None, None, 4096)] 0                                            
__________________________________________________________________________________________________
dec_embd (Embedding)            (None, None, 100)    214000      dec_input[0][0]                  
__________________________________________________________________________________________________
enc_lstm (LSTM)                 [(None, 100), (None, 1678800     enc_input[0][0]                  
_______________________________________________________________________________________

In [17]:
plot_model(model, show_shapes=True)


('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [38]:
batch_size = 128
epochs = 10

model.fit(x=[train_features,padded], y=target_padded, batch_size=batch_size, epochs=epochs)

Epoch 1/10
25/25 [==============================] - 25s 992ms/step - loss: 1.2493
Epoch 2/10
25/25 [==============================] - 49s 2s/step - loss: 1.2317
Epoch 3/10
25/25 [==============================] - 43s 2s/step - loss: 1.2150
Epoch 4/10
25/25 [==============================] - 48s 2s/step - loss: 1.1993
Epoch 5/10
25/25 [==============================] - 41s 2s/step - loss: 1.1843
Epoch 6/10
25/25 [==============================] - 36s 1s/step - loss: 1.1698
Epoch 7/10
25/25 [==============================] - 38s 2s/step - loss: 1.1557
Epoch 8/10
25/25 [==============================] - 35s 1s/step - loss: 1.1417
Epoch 9/10
25/25 [==============================] - 34s 1s/step - loss: 1.1276
Epoch 10/10
25/25 [==============================] - 32s 1s/step - loss: 1.1127


((4207, 38), (4207, 38))

In [39]:
encoder_model_inf = Model(enc_input, enc_states)

dec_inf_input_h = Input((lstm_units,))
dec_inf_input_c = Input((lstm_units,))
dec_inf_input_states = [dec_inf_input_h, dec_inf_input_c]

dec_inf_out, dec_inf_h, dec_inf_c = dec_lstm(dec_embedding(dec_input),
                                             initial_state=dec_inf_input_states)
dec_inf_states = [dec_inf_h, dec_inf_c]
dec_inf_output = dec_dense(dec_inf_out)

decoder_model_inf = Model(inputs=[dec_input] + dec_inf_input_states,
                          outputs=[dec_inf_output] + dec_inf_states)

In [40]:
encoder_model_inf.summary()
plot_model(encoder_model_inf, show_shapes=True)

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
enc_input (InputLayer)       [(None, None, 4096)]      0         
_________________________________________________________________
enc_lstm (LSTM)              [(None, 100), (None, 100) 1678800   
Total params: 1,678,800
Trainable params: 1,678,800
Non-trainable params: 0
_________________________________________________________________
('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [41]:
decoder_model_inf.summary()
plot_model(decoder_model_inf, show_shapes=True)

Model: "functional_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dec_input (InputLayer)          [(None, None)]       0                                            
__________________________________________________________________________________________________
dec_embd (Embedding)            (None, None, 100)    214000      dec_input[0][0]                  
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 100)]        0                                            
______________________________________________________________________________________

In [42]:
word_index = tokenizer.word_index
reverse_word_index = dict([(v,k) for k,v in word_index.items()])

In [45]:
def generate(input_seq):

    states_val = encoder_model_inf.predict(input_seq)
    target_seq = [tokenizer.word_index['\t']]

    predicted_sent = []
    stop_condition = False

    while not stop_condition:

        decoder_out, decoder_h, decoder_c = decoder_model_inf.predict(
            x=[np.array(target_seq)] + states_val)
        max_val_index = np.argmax(decoder_out[0, 0])

        if reverse_word_index[max_val_index] == '\n' or max_val_index == 0 or len(predicted_sent) > max_len:
            stop_condition = True

        target_seq = [max_val_index]
        predicted_sent.append(reverse_word_index[max_val_index])
        states_val = [decoder_h, decoder_c]

    return " ".join(predicted_sent)

In [51]:
index=100
generate(np.array([train_features[index]])), tokenizer.sequences_to_texts([padded[index]])

('is is is \n', ['\t the rabbit is eating \n'])

['\t woman gallops on horse \n']